In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM

In [3]:
import json
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [58]:
import json
with open('landmarks.json', 'r') as f:
    landmarks = json.load(f)
landmarks.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

In [59]:
len(landmarks['0'][0])

4356

In [60]:
with open('length.json', 'r') as f:
    length = json.load(f)
length.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

In [61]:
max_length=max(length.values())
max_length

5940

In [62]:
Y=pd.read_csv('labels.csv')
Y.tail()

,Class Label
717,16
718,16
719,16
720,16
721,16


In [63]:
f=open('words.txt','r')
words=f.read().split('\n')
words

['Boy',
 'Can',
 'Eat',
 'Fine',
 'Girl',
 'Help',
 'How',
 'Hungry',
 'I',
 'Mother',
 'Namaste',
 'Name',
 'Parents',
 'Sister',
 'Sleep',
 'This',
 'You',
 '']

In [68]:
# X = pd.DataFrame(columns=['Landmarks'])
X=[]

In [69]:
def pad_data():
    for i in landmarks:
        landmarks[i][0].extend([0]*(max_length-length[i]))
        print(len(landmarks[i][0]))
        print(landmarks[i])
        X.append(np.array(landmarks[i]))
        # X['Landmarks'].append(np.array(landmarks[i]))
        # print(a.shape)

In [70]:
pad_data()

7524
7128
7128
7524
7524
7722
7722
7722
7920
8514
8316
8514
7920
8118
8118
8118
7524
8118
8118
8118
7722
8316
8316
8316
8514
8514
8118
8316
8316
8316
8316
8316
8514
8514
8514
8514
7128
7128
7128
7128
8118
8514
8514
8316
8316
8118
8118
8316
7722
7920
7920
7722
7128
7128
7128
8316
7920
7920
8316
6930
6732
6732
6930
9108
8118
8118
8712
6534
6138
6138
6534
8316
7920
7920
8118
7920
7524
7524
7326
6534
6534
6732
7326
7326
7326
8316
8316
8514
7920
7920
8118
7128
7128
7128
7128
7524
7722
7524
7524
7920
7722
7722
7920
6732
6534
6534
6732
7722
7722
7722
7722
6732
6930
6732
6930
7524
7326
7326
7326
7326
7128
7128
7326
6930
6732
6732
7128
6930
7128
7128
6930
7326
7128
6732
7326
7524
7722
7524
7524
8316
8316
8118
8316
7524
7722
7524
7524
6930
7128
6930
6930
8316
8316
8118
8316
7722
8118
7920
7920
8514
8316
8316
8316
7722
7524
7722
7920
7722
7920
7722
7524
7722
7920
7920
8316
7722
7722
7722
7920
7920
8118
7920
8118
7326
7326
7326
7326
7524
7128
7128
7326
7722
7722
7722
7722
8910
7722
7722
7920
7128


In [67]:
len(X)

722

In [56]:
X=pd.Series(X, name='Landmarks')
X

0      [[-0.2904877959721088, -0.16641040988436406, -...
1      [[-1.142779883255223, -1.142779883255223, -1.1...
2      [[-1.0683707317528615, -1.0683707317528615, -1...
3      [[-0.22918893763408849, -0.11131902349273194, ...
4      [[-0.8761141557032232, -0.8000427084121983, -0...
                             ...                        
717    [[-0.027518266807864268, 0.10758721437182653, ...
718    [[-0.3086242130735477, -0.18891817698197738, -...
719    [[-0.08535734973601969, -0.04579855174747038, ...
720    [[-0.12314708994644989, 0.008995784117713651, ...
721    [[-0.23017973876876122, -0.118937528194917, -0...
Name: Landmarks, Length: 722, dtype: object

In [57]:
X[0].shape

(1, 15444)

In [12]:
Y=to_categorical(Y).astype(int)
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
#X has to be of Series
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Model-1

In [ ]:
model=Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(62, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[tb_callback])

In [ ]:
model.summary()

MODEL 2

In [ ]:
model=Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [ ]:
res=model.predict(x_test)
res[0]

##Save model

In [ ]:
model.save('model.h5')

model=tf.keras.models.load_model('model.h5')

Prediction

In [ ]:
words[np.argmax(model.predict(x_test)[0])]

In [ ]:
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
y_pred=model.predict(x_test)

In [ ]:
y_true=np.argmax(y_test, axis=1).tolist()
y_pred=np.argmax(y_pred, axis=1).tolist()

In [ ]:
y_true

In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true, y_pred)